In [55]:
import pandas as pd
from pymongo import MongoClient
import re
import csv


### Connection avec mongodb


In [56]:
client = MongoClient("localhost:27017")
database = client.scrapyPipeline
movies = database.movies
serie = database.serie
# movies.drop()
# serie.drop()

### Nettoyage des données

In [57]:
# df_movie = pd.read_csv("csv/items.csv")
# df_serie = pd.read_csv("csv/serie.csv")

In [58]:
# df_movie["titre_original"].replace(to_replace="Original title: ", value='', regex=True,inplace=True)


In [59]:
# df_movie["duree"].replace(to_replace=",", value='', regex=True,inplace=True)
# df_serie["duree"].replace(to_replace=",", value='', regex=True,inplace=True)
def convertion_duree(s):
    
    split_cell = re.split("\s", s)
    duree = 0
    for elt in split_cell:
        if "h" in elt:
            duree = int(re.split("h", elt)[0]) *60 +duree
        elif "m" in elt:
            duree =int(re.split("m", elt)[0]) +duree
    return duree
    




# df_movie['duree'] = df_movie['duree'].apply(lambda x :  convertion_duree(str(x)))
# df_serie['duree'] = df_serie['duree'].apply(lambda x :  convertion_duree(str(x)))



In [60]:
# df_movie.to_csv("csv/items.csv",index=False)
# df_serie.to_csv("csv/serie.csv",index=False)

### Import des données dans mongodb

In [61]:
# header = ["acteurs","date","descriptions","duree","genre","pays","public","score","titre","titre_original","url"]
# csvfile = open('csv/items.csv', 'r')
# reader = csv.DictReader( csvfile )

# for each in reader:
#     row={}
#     for field in header:

#         if field in  ["acteurs","genre","pays"]:
#             row[field]=each[field].split(",")
#         elif field in ["date","duree"]:
#             row[field]=int(each[field])
#         elif field =="score":
#             row[field]=float(each[field])
#         else :
#             row[field]=each[field]
        
#     movies.insert_one(row)

In [62]:
# header = ["acteurs","date","descriptions","duree","genre","pays","public","score","titre","titre_original"]
# csvfile = open('csv/serie.csv', 'r')
# reader = csv.DictReader( csvfile )

# for each in reader:
#     row={}
#     for field in header:

#         if field in  ["acteurs","genre","pays"]:
#             row[field]=each[field].split(",")
#         elif field in ["duree"]:
#             row[field]=int(each[field])
#         elif field =="score":
#             row[field]=float(each[field])
#         else :
#             row[field]=each[field]  
        
#     serie.insert_one(row)

## question n°1

In [63]:
movies.find().sort("duree",-1).limit(1)[0]

{'_id': ObjectId('639c6900a922183b556d3b3a'),
 'titre': 'Gone with the Wind',
 'titre_original': '',
 'score': 8.2,
 'genre': ['Drama', 'Romance', 'War'],
 'date': 1939,
 'duree': 238,
 'descriptions': 'The manipulative daughter of a Georgia plantation owner conducts a turbulent romance with a roguish profiteer during the American Civil War and Reconstruction periods.',
 'acteurs': ['Clark Gable', 'Vivien Leigh', 'Thomas Mitchell'],
 'public': 'Passed',
 'pays': ['United States'],
 'url': 'https://m.media-amazon.com/images/M/MV5BYjUyZWZkM2UtMzYxYy00ZmQ3LWFmZTQtOGE2YjBkNjA3YWZlXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_QL75_UX190_CR0,5,190,281_.jpg',
 'langue': ['English']}

## Question n°2

In [64]:
five_movies=movies.find().sort("score",-1).limit(5)
for movie in five_movies:
    print(movie["titre"])

The Shawshank Redemption
The Godfather
The Godfather Part II
Schindler's List
12 Angry Men


## Question n°3


In [65]:
Freeman_movies = movies.find({"acteurs":{"$in":["Morgan Freeman"]}})
for movie in Freeman_movies:
    print(movie["titre"])
count=movies.find({"acteurs":{"$in":["Morgan Freeman"]}}).count()
print(count)

The Shawshank Redemption
Se7en
Million Dollar Baby
Unforgiven
4


/tmp/ipykernel_35554/2969739726.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  count=movies.find({"acteurs":{"$in":["Morgan Freeman"]}}).count()


In [66]:
Cruise_movies = movies.find({"acteurs":{"$in":["Tom Cruise"]}})
for movie in Cruise_movies:
    print(movie["titre"])
count=movies.find({"acteurs":{"$in":["Tom Cruise"]}}).count()
print(count)

Top Gun: Maverick
1


/tmp/ipykernel_35554/432494079.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  count=movies.find({"acteurs":{"$in":["Tom Cruise"]}}).count()


## Question n°4

In [67]:
horror_movies = movies.find({"genre":{"$in":["Horror"]}}).sort("score",-1).limit(3)
for movie in horror_movies:
    print(movie["titre"])

Psycho
Alien
The Shining


In [68]:
drama_movies = movies.find({"genre":{"$in":["Drama"]}}).sort("score",-1).limit(3)
for movie in drama_movies:
    print(movie["titre"])

The Shawshank Redemption
The Godfather
12 Angry Men


In [69]:
comedy_movies = movies.find({"genre":{"$in":["Comedy"]}}).sort("score",-1).limit(3)
for movie in comedy_movies:
    print(movie["titre"])

Life Is Beautiful
Modern Times
The Intouchables


## Question 6

In [80]:
american_movies = movies.find().sort("score",-1).limit(100)
count=0
for movie in american_movies:
    if "United States" in movie["pays"]:
        count +=1

print(count)

74


In [71]:
liste_movies = movies.find().sort("score",-1).limit(100)
count=0
for movie in liste_movies:
    if "France" in movie["pays"]:
        count +=1
pourcentage = count
print(pourcentage)

11


In [81]:
top_100 = movies.find().sort("score",-1).limit(100)
france = 0
us = 0
for movie in top_100:
    if "France" in movie["pays"]:
        france += 1
    if "United States" in movie["pays"]:
        us += 1
f"Dans le Top 100 des films : US ({us}%) FR ({france}%)"

'Dans le Top 100 des films : US (74%) FR (11%)'

## Question n°7


In [73]:
liste_genre =[]
list_movies = movies.find()
for movie in list_movies:
    genre_movie = movie["genre"]
    for genre in genre_movie:
        if not genre in liste_genre:
            liste_genre.append(genre)
print(liste_genre, len(liste_genre))

['Drama', 'Action', 'Sci-Fi', 'Biography', 'Crime', 'Mystery', 'Family', 'Fantasy', 'Thriller', 'Adventure', 'Western', 'Romance', 'History', 'Animation', 'Comedy', 'Film-Noir', 'War', 'Horror', 'Sport', 'Music', 'Musical'] 21


In [78]:
dict_temps_genre={}
for genre in liste_genre:
    temps_total = 0
    list_movies = movies.find({"genre":{"$in":[genre]}})
    nb_movies=list_movies.count()
    
    for movie in list_movies:
        temps_total = movie["duree"] + temps_total

    dict_temps_genre[genre] =  temps_total/nb_movies
print(dict_temps_genre)


{'Drama': 134.63636363636363, 'Action': 133.92156862745097, 'Sci-Fi': 131.7, 'Biography': 147.06896551724137, 'Crime': 131.7843137254902, 'Mystery': 124.61290322580645, 'Family': 114.53846153846153, 'Fantasy': 123.57142857142857, 'Thriller': 121.03333333333333, 'Adventure': 129.41666666666666, 'Western': 149.33333333333334, 'Romance': 118.47826086956522, 'History': 156.0, 'Animation': 100.95652173913044, 'Comedy': 106.32608695652173, 'Film-Noir': 110.0, 'War': 140.43478260869566, 'Horror': 120.6, 'Sport': 122.0, 'Music': 134.25, 'Musical': 103.0}


/tmp/ipykernel_35554/3980943192.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  nb_movies=list_movies.count()


In [75]:
avg_genre= (movies.aggregate([
    {"$unwind":"$genre"},
        {"$group": {
            "_id": "$genre","duree": { "$avg": "$duree" }
        }
    }
]))
for i in avg_genre:
    print(i)

{'_id': 'Thriller', 'duree': 121.03333333333333}
{'_id': 'Fantasy', 'duree': 123.57142857142857}
{'_id': 'Musical', 'duree': 103.0}
{'_id': 'Film-Noir', 'duree': 110.0}
{'_id': 'Animation', 'duree': 100.95652173913044}
{'_id': 'War', 'duree': 140.43478260869566}
{'_id': 'Romance', 'duree': 118.47826086956522}
{'_id': 'Action', 'duree': 133.92156862745097}
{'_id': 'Comedy', 'duree': 106.32608695652173}
{'_id': 'Horror', 'duree': 120.6}
{'_id': 'Drama', 'duree': 134.63636363636363}
{'_id': 'Sci-Fi', 'duree': 131.7}
{'_id': 'Music', 'duree': 134.25}
{'_id': 'Crime', 'duree': 131.7843137254902}
{'_id': 'Adventure', 'duree': 129.41666666666666}
{'_id': 'History', 'duree': 156.0}
{'_id': 'Biography', 'duree': 147.06896551724137}
{'_id': 'Family', 'duree': 114.53846153846153}
{'_id': 'Mystery', 'duree': 124.61290322580645}
{'_id': 'Sport', 'duree': 122.0}
{'_id': 'Western', 'duree': 149.33333333333334}


In [76]:
for key,value in dict_temps_genre.items():
    print(f"Le temps moyen du genre {key} est {round(value,2)} minutes ")

Le temps moyen du genre Drama est 134.64 minutes 
Le temps moyen du genre Action est 133.92 minutes 
Le temps moyen du genre Sci-Fi est 131.7 minutes 
Le temps moyen du genre Biography est 147.07 minutes 
Le temps moyen du genre Crime est 131.78 minutes 
Le temps moyen du genre Mystery est 124.61 minutes 
Le temps moyen du genre Family est 114.54 minutes 
Le temps moyen du genre Fantasy est 123.57 minutes 
Le temps moyen du genre Thriller est 121.03 minutes 
Le temps moyen du genre Adventure est 129.42 minutes 
Le temps moyen du genre Western est 149.33 minutes 
Le temps moyen du genre Romance est 118.48 minutes 
Le temps moyen du genre History est 156.0 minutes 
Le temps moyen du genre Animation est 100.96 minutes 
Le temps moyen du genre Comedy est 106.33 minutes 
Le temps moyen du genre Film-Noir est 110.0 minutes 
Le temps moyen du genre War est 140.43 minutes 
Le temps moyen du genre Horror est 120.6 minutes 
Le temps moyen du genre Sport est 122.0 minutes 
Le temps moyen du genre